In [4]:
import numpy as np
import Utils as ut

/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [ ]:
from sklearn import datasets
# make_classification(n_samples=100, n_features=20, n_informative=2, n_redundant=2, n_repeated=0, 
#                     n_classes=2, n_clusters_per_class=2, weights=None, flip_y=0.01, class_sep=1.0, 
#                     hypercube=True, shift=0.0, scale=1.0, shuffle=True, random_state=None)
ds, label = datasets.make_classification(n_samples=300, n_features=30, n_informative=10, n_redundant=0, n_classes=2)

In [ ]:
np.where(label==0)
label[np.where(label==0)]=-1

In [ ]:
def random_sampling(X, y, n_samples):
    """Extract n_samples from X and y."""
    from sklearn.model_selection import StratifiedShuffleSplit
    
    return next(StratifiedShuffleSplit(n_splits=1, test_size=n_samples).split(X, y))
     

In [ ]:
tr_idx, ts_idx = random_sampling(ds, label, n_samples=0.25)


ds1 = np.hstack([ds[:,:5], ds[:,10:15]])
ds1_tr = ds1[tr_idx]
ds1_ts = ds1[ts_idx]

ds2 = np.hstack([ds[:,5:10], ds[:,15:20]])
ds2_tr = ds2[tr_idx]
ds2_ts = ds2[ts_idx]

ds3 = ds[:,20:]
ds3_tr = ds3[tr_idx]
ds3_ts = ds3[ts_idx]

l_tr = label[tr_idx]
l_ts = label[ts_idx]

In [ ]:
kernel_types = ['linear', 'polynomial', 'gaussian']
k_dataset_wrapper = []
k_train_list = []
k_test_list = []

for train, test in zip([ds1_tr, ds2_tr, ds3_tr], [ds1_ts, ds2_ts, ds3_ts]):
    for k_type in kernel_types:
        if k_type=='polynomial': 
            current = ut.kernel(train, K_type=k_type, param=10)
        else:
            current = ut.kernel(train, K_type=k_type, param=2)
            
        k_dataset_wrapper.append({'kernel':current, 'train_ds':train, 'test_ds':test})
        k_train_list.append(current.kernelMatrix(train))
        k_test_list.append(current.kernelMatrix(test))


In [ ]:
#idealK_overTr = np.dot(l_tr.reshape(-1,1), l_tr.reshape(-1,1).T)
new_kernels_config = ut.parameterOptimization(k_dataset_wrapper, l_tr, n_epoch=100, tol=0.0001, verbose=True)

In [ ]:
weights = ut.centeredKernelAlignment(k_train_list, l_tr)
print(weights)

In [ ]:
kernel_type = {'linear': [0], 'polynomial': [2,3,4], 'gaussian':[0.3, 0.5, 0.7]}
eta, ca, params = ut.centeredKernelAlignmentCV(kernel_type, [ds1_tr, ds2_tr, ds3_tr], l_tr)

In [ ]:
pred = np.zeros(len(ts_idx))
for k_test, w in zip(k_test_list, weights):
    weighted_labeled_kernel = np.multiply(k_test, l_tr*w)  
    pred += np.sum(weighted_labeled_kernel, axis=1)
pred = np.sign(pred)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(l_ts, pred)

In [ ]:
a = np.array([[1,2],[3,4]])
print(a.ravel().reshape(-1,1).shape)